# Stage 11 — Evaluation & Risk Communication

"This notebook implements the homework steps: baseline fit, bootstrap CI, scenario comparisons,
subgroup diagnostics, visualization, and a stakeholder-facing write-up. It is self-contained and
uses `/data/data_stage11_eval_risk.csv` if available otherwise it generates synthetic data.


!pip install matplotlib pandas numpy datetime jupyterlab python-dotenv seaborn scikit-learn missingno pyarrow fastparquet

In [76]:
import os
import numpy as np
import pandas as pd
from pathlib import Path 
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import seaborn as sns

np.random.seed(3)

# plotting defaults
plt.rcParams['figure.figsize'] = (8,5)

In [88]:
data_dir = Path('data')
data_dir.mkdir(exist_ok=True)

csv_path = data_dir / 'data_stage11_eval_risk.csv'

if csv_path.exists():
    try:
        df = pd.read_csv(csv_path, parse_dates=['date'])
    except ValueError as e:
        print(f"Error reading CSV: {e}\nRegenerating synthetic data...")
        regenerate = True
    else:
        regenerate = False
else:
    regenerate = True

if regenerate:
    n = 180
    dates = pd.date_range('2022-06-01', periods=n, freq='D')
    seg = np.random.choice(['A','B','C'], size=n, p=[0.5,0.3,0.2])
    x = np.linspace(0, 9, n) + np.random.normal(0, 0.7, n)
    y = 2.1 * x + 0.8 + np.random.standard_t(df=3, size=n) * 1.1
    x[np.random.choice(np.arange(n), size=round(0.05*n), replace=False)] = np.nan
    df = pd.DataFrame({'date': dates, 'segment': seg, 'x_feature': x, 'y_target': y})
    df.to_csv(csv_path, index=False)

df.head()

,date,segment,x_feature,y_target
0,2022-06-01,B,0.298177,1.141263
1,2022-06-02,B,-0.054030,-1.199527
2,2022-06-03,A,0.685649,2.416025
3,2022-06-04,B,0.495321,0.764932
4,2022-06-05,C,-0.402499,0.800794


In [89]:
import numpy as np

# --- 1. Imputation functions ---
def mean_impute(a: np.ndarray) -> np.ndarray:
    """Replace missing values (NaN) with the column mean."""
    m = np.nanmean(a)            # Mean of non-NaN values
    out = a.copy()                # Copy original array
    out[np.isnan(out)] = m        # Replace NaN with mean
    return out

def median_impute(a: np.ndarray) -> np.ndarray:
    """Replace missing values (NaN) with the column median."""
    m = np.nanmedian(a)
    out = a.copy()
    out[np.isnan(out)] = m
    return outs


# --- 2. Simple Linear Regression ---
class SimpleLinReg:
    """Very basic linear regression using least squares."""
    def fit(self, X, y):
        # Add intercept column: [1, X]
        X1 = np.c_[np.ones(len(X)), X.ravel()]
        # Solve for coefficients using pseudo-inverse
        beta = np.linalg.pinv(X1) @ y
        self.intercept_ = float(beta[0])       # Intercept term
        self.coef_ = np.array([float(beta[1])]) # Slope coefficient
        return self

    def predict(self, X):
        # Prediction: y = intercept + coef * X
        return self.intercept_ + self.coef_[0] * X.ravel()


# --- 3. Error Metric ---
def mae(y_true, y_pred):
    """Mean Absolute Error (MAE) between true and predicted values."""
    return float(np.mean(np.abs(y_true - y_pred)))


# --- 4. Bootstrap for Uncertainty ---
def bootstrap_metric(y_true, y_pred, fn, n_boot=500, seed=111, alpha=0.05):
    """
    Bootstrap confidence intervals for any metric function.
    
    y_true: true values
    y_pred: model predictions
    fn: metric function (e.g., MAE)
    n_boot: number of bootstrap samples
    alpha: confidence level (e.g., 0.05 → 95% CI)
    """
    rng = np.random.default_rng(seed)
    idx = np.arange(len(y_true))
    stats = []

    for _ in range(n_boot):
        # Sample with replacement
        b = rng.choice(idx, size=len(idx), replace=True)
        stats.append(fn(y_true[b], y_pred[b]))

    # CI bounds
    lo, hi = np.percentile(stats, [100*alpha/2, 100*(1-alpha/2)])
    return {'mean': float(np.mean(stats)), 'lo': float(lo), 'hi': float(hi)}


# --- 5. Wrappers for training & prediction ---
def fit_fn(X, y):
    """Fit SimpleLinReg on (X, y)."""
    return SimpleLinReg().fit(X, y)

def pred_fn(model, X):
    """Get predictions from model."""
    return model.predict(X)


In [104]:
from src.evaluation import SimpleLinReg
print(SimpleLinReg.__module__)
print(hasattr(SimpleLinReg, "fit"))  #check if fit exist

"WHY CANNOT FIND fit"

src.evaluation
False


'WHY CANNOT FIND fit'

In [101]:
from src.evaluation import fit_fn
from src.evaluation import mean_impute, median_impute, fit_fn, pred_fn, mae, bootstrap_metric, SimpleLinReg

_base = mean_impute(df['x_feature'].values)
model = fit_fn(X_base.reshape(-1,1), df['y_target'].values)
y_hat = pred_fn(model, X_base.reshape(-1,1))
print(mae(df['y_target'].values, y_hat))


X_raw = df['x_feature'].values      # Get raw feature values (may have NaNs)
y = df['y_target'].values           # Get target values
X_base = mean_impute(X_raw)         # Impute missing values with mean
model = fit_fn(X_base.reshape(-1,1), y)  # Fit SimpleLinReg on 2D input (n x 1)
y_hat = model.predict(X_base.reshape(-1,1)) # Predict on the same data
df['x_imputed'] = X_base            # Save imputed feature into dataframe
base_mae = mae(y, y_hat)            # Compute mean absolute error
base_mae                           # Print or return MAE

AttributeError: 'SimpleLinReg' object has no attribute 'fit'

In [100]:
# --- 1. Prepare data and model prediction ---
resid = y - y_hat                 # residuals = actual - predicted
sigma_hat = np.std(resid, ddof=1) # standard deviation of residuals
n = len(y)
se_mean = sigma_hat / np.sqrt(n)  # standard error of mean prediction

# Make a smooth x-grid and predict values
x_grid = np.linspace(np.nanmin(X_base), np.nanmax(X_base), 100).reshape(-1,1)
pred_line = model.predict(x_grid)

# Parametric 95% confidence interval: prediction ± 1.96*SE
gauss_lo = pred_line - 1.96 * se_mean
gauss_hi = pred_line + 1.96 * se_mean


# --- 2. Bootstrap confidence interval ---
def bootstrap_predictions(X, y, x_grid, n_boot=500, seed=111):
    rng = np.random.default_rng(seed)
    preds = []
    n = len(y)

    for _ in range(n_boot):
        # Sample with replacement
        sample_idx = rng.choice(n, size=n, replace=True)
        # Fit model on the sampled data
        m = fit_fn(X[sample_idx].reshape(-1,1), y[sample_idx])
        # Store predictions on grid
        preds.append(m.predict(x_grid))

    preds = np.vstack(preds)
    # Return mean prediction and 95% interval
    return preds.mean(axis=0), np.percentile(preds, 2.5, axis=0), np.percentile(preds, 97.5, axis=0)

# Run bootstrap
m_boot, lo_boot, hi_boot = bootstrap_predictions(X_base, y, x_grid, n_boot=300)


# --- 3. Plot everything ---
plt.figure(figsize=(6,4)) 
plt.scatter(X_base, y, alpha=0.3, label="Data") #Creaste a scatter plot
plt.plot(x_grid, pred_line, 'r', label="Prediction")
plt.fill_between(x_grid.ravel(), gauss_lo, gauss_hi, alpha=0.2, label="Gaussian CI")
plt.fill_between(x_grid.ravel(), lo_boot, hi_boot, alpha=0.2, label="Bootstrap CI")
plt.legend()
plt.title("Parametric vs Bootstrap Confidence Intervals")
plt.show()

NameError: name 'y_hat' is not defined